In [1]:
import pandas, numpy, math, pickle, urllib, json

def get_driving_distance_osm(str_lat_comma_long1,str_lat_comma_long2):
    query_args = { 'fromPlace':str_lat_comma_long1, 'toPlace':str_lat_comma_long2,'mode':'CAR' }
    encoded_args = urllib.urlencode(query_args)
    #print 'Encoded:', encoded_args
    url = 'http://13.218.151.99:8080/otp/routers/default/plan?' + encoded_args
    temp = json.load(urllib.urlopen(url))
    return temp['plan']['itineraries'][0]['legs'][0]['distance']

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a)) 
    km = 6367 * c
    return km

#Distances in Simulated data
import numpy, pickle
from profit_maximization import euclidean
flag_sim_dist_stat = False
if flag_sim_dist_stat:
    data = pickle.load(open('../../../Xharecost_MS_annex/bias_data.pkl','rb'))
    instance = data['instances'][100]
    all_requests = instance['all_requests']
    distances = numpy.zeros(len(all_requests))
    for i in all_requests:
        distances[i] = euclidean(all_requests[i]['orig'],all_requests[i]['dest'])
    distDF = pandas.DataFrame(data=distances,index=range(len(all_requests)),columns=['dist'])
    distDF.describe()

In [2]:
df0 = pandas.read_csv('../../../Xharecost_MS_annex/hour9.csv',usecols= ['pickup_datetime','passenger_count','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude'])

df = df0[(abs(df0.pickup_latitude-0) > 1e-5) & \
         (abs(df0.pickup_longitude-0) > 1e-5) & \
         (abs(df0.dropoff_latitude-0) > 1e-5) & \
         (abs(df0.dropoff_longitude-0) > 1e-5)]
#print df.describe()

In [5]:
flag_debug = False
if flag_debug:
    str_lat_comma_long1 = '40.616276,-74.017921'
    str_lat_comma_long2 = '40.865898,-73.697060'
    print get_driving_distance_osm(str_lat_comma_long1,str_lat_comma_long2)

In [10]:
df['pickup_datetime'] = pandas.to_datetime(df['pickup_datetime'])
for i in range(60):
    df1 = df[(df['pickup_datetime'].dt.minute >= i) & (df['pickup_datetime'].dt.minute < i+1)]
    print 'minute: {0} count: {1}'.format(i,len(df1))

minute: 0 count: 213
minute: 1 count: 254
minute: 2 count: 230
minute: 3 count: 231
minute: 4 count: 192
minute: 5 count: 227
minute: 6 count: 224
minute: 7 count: 211
minute: 8 count: 204
minute: 9 count: 227
minute: 10 count: 243
minute: 11 count: 225
minute: 12 count: 249
minute: 13 count: 197
minute: 14 count: 207
minute: 15 count: 218
minute: 16 count: 198
minute: 17 count: 223
minute: 18 count: 229
minute: 19 count: 223
minute: 20 count: 224
minute: 21 count: 214
minute: 22 count: 209
minute: 23 count: 201
minute: 24 count: 233
minute: 25 count: 211
minute: 26 count: 226
minute: 27 count: 201
minute: 28 count: 208
minute: 29 count: 196
minute: 30 count: 219
minute: 31 count: 188
minute: 32 count: 200
minute: 33 count: 215
minute: 34 count: 211
minute: 35 count: 210
minute: 36 count: 187
minute: 37 count: 208
minute: 38 count: 212
minute: 39 count: 184
minute: 40 count: 223
minute: 41 count: 179
minute: 42 count: 207
minute: 43 count: 213
minute: 44 count: 216
minute: 45 count: 22

In [ ]:
len(df1)

In [2]:
from profit_maximization import *
get_instance_params()['GAMMA_ARRAY']

[0, 0.1, 0.2, 0.3, 0.4]